# 1. Installing Dependencies

!pip install -q datasets transformers accelerate timm
!pip install -q -U albumentations>=1.4.5 torchmetrics pycocotools

In [ ]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim import Adam, SGD
import torch.nn.functional as F

import torchvision
from torchvision.transforms import transforms
from torchvision.models import models

# Building and Playing with DATA!

from roboflow import Roboflow
rf = Roboflow(api_key="here")
project = rf.workspace("roboflow-100").project("uno-deck")
version = project.version(2)
dataset = version.download("yolov8")

In [ ]:
img_path = '/content/uno.jpg'


model_1 = YOLO('yolov8n.pt')
model_2 = YOLO('yolov8n.pt')
model_3 = YOLO('yolov8n.pt')
model_4 = YOLO('yolov8n.pt')


img = cv2.imread(img_path) 

result_1 = model_1(img, stream = True) 
result_2 = model_2(img, stream = True) 
result_3 = model_3(img, stream = True) 
result_4 = model_4(img, stream = True)